In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = 'data/amplitude_simulations/'

In [55]:
baseline_lux_values = np.array([0, 25, 50, 75, 100, 250, 500, 750, 1000], dtype=float)
pulse_at_values = np.linspace(0, 14, 17)

simulation_days = 20
dt = 0.001
last_day_start = int((simulation_days - 1) * 24 / dt)
day_length_in_idx = int(24.0 / dt) - 1
time_array = np.arange(0, simulation_days * 24, dt)  # Time array for the entire simulation

avg_amplitude = np.zeros((len(baseline_lux_values), len(pulse_at_values)), dtype=float)

for idx, baseline_lux in enumerate(baseline_lux_values):
    for jdx, pulse_at in enumerate(pulse_at_values):
        data = np.load(f'{data_path}baseline_{baseline_lux}_pulse_at_{pulse_at}.npz')
        sol = data['sol']
        light = data['light_history']
        # Calculate amplitude after pulse
        time_last_day = time_array[last_day_start:] - time_array[last_day_start]
        x_last_day = sol[0, last_day_start:]
        xc_last_day = sol[1, last_day_start:]
        light_last_day = light[last_day_start:]
        amplitude_last_day = np.sqrt(x_last_day ** 2 + xc_last_day ** 2)
        avg_amplitude[idx, jdx] = np.mean(amplitude_last_day)

# Save average amplitude
avg_amplitude_df = pd.DataFrame(avg_amplitude, index=baseline_lux_values, columns=pulse_at_values)
avg_amplitude_df.index.name = 'Baseline Lux'
avg_amplitude_df.columns.name = 'Pulse At (hours)'
avg_amplitude_df.to_csv(f'{data_path}average_amplitude.csv')